In [22]:
from tinysmpc import VirtualMachine, PrivateScalar, SharedScalar

# Private 2 Party State Machine

This notebook demonstrates how a state machine can be privately evaluated using two non-colluding servers. For this protocol "privacy" is defined as:
- The **servers** do not learn the **input** of the client
- The **servers and client** do not know the **current state** of the machine (apart from the initial state)

## General Protocol Overview

<!-- TODO: add image here -->

One time operation:
1. Regex is converted into a state machine
2. State machine is converted into polynomial representation over a finite field (or ring)
3. Create arithmetic circuit for polynomial representation and share with servers

Each time the client wants to evaluate the state machine:
1. Client inputs a token (e.g. a character)
2. The token is additively secret shared with servers ($token_0$ shared with $server_0$ and $token_1$ shared with $server_1$, where $token_0 + token_1 = input$)
3. Servers evaluate the arithmetic circuit using their own individual input share as well as their share of the current state
4. If more input go to step 1

## 1. Regex to State Machine

For this demo we will be basing our state machine off the regex expression "ab". This machine will only accept the string "ab" and nothing else. The state machine can be visualized as follows:

![transition graph of state machine for regex "ab"](./images/ab_transition_graph.png)
<!-- Generated using https://ivanzuzak.info/noam/webapps/fsm_simulator/ -->

### State Table for "ab"

|             | **INPUT 'a'** | **INPUT 'b'** |
| ----------- | ------------- | ------------- |
| **STATE 0** | STATE 2       | STATE 3       |
| **STATE 1** | STATE 3       | STATE 3       |
| **STATE 2** | STATE 3       | STATE 1       |
| **STATE 3** | STATE 3       | STATE 3       |

## 2. State Machine to Polynomial Representation

To make interpolation simpler we can the state transition table as a bunch of 3D points.

```python
# (STATE, INPUT ('a'=0 and 'b'=1), NEXT STATE)
(0, 0, 2)
(1, 0, 3)
(2, 0, 3)
(3, 0, 3)
(0, 1, 3)
(1, 1, 3)
(2, 1, 1)
(3, 1, 3)
```

We can now use bilinear interpolation to construct this equation:

$$f(x, y) = 2 + 1y + 10922x + 54602xy + 65520x^2 + 65518x^2y + 54601x^3 + 10921x^3y$$

You have to trust me, but this equation is the polynomial representation of the state machine above over a Galois ring where $p = 65521$. 

In [23]:
def f(x, y):
    return 3 * x * y + x + y + 4

ns = 0

token = 3
ns = f(token, ns)
print(ns)

token = 5
ns = f(token, ns)
print(ns)

token = 1
ns = f(token, ns)
print(ns)

7
121
489


In [24]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
charlie = VirtualMachine('charlie')

ns_a = PrivateScalar(0, alice)
ns_b = PrivateScalar(0, bob)

shared_ns = ns_a.share([alice, bob])

token = PrivateScalar(3, charlie)
shared_token = token.share([alice, bob])
shared_ns = f(shared_token, shared_ns)
print(shared_ns.reconstruct(alice))

token = PrivateScalar(5, charlie)
shared_token = token.share([alice, bob])
shared_ns = f(shared_token, shared_ns)
print(shared_ns.reconstruct(alice))

token = PrivateScalar(1, charlie)
shared_token = token.share([alice, bob])
shared_ns = f(shared_token, shared_ns)
print(shared_ns.reconstruct(alice))

PrivateScalar(7, 'alice')
PrivateScalar(121, 'alice')
PrivateScalar(489, 'alice')


In [25]:
PRIME = 65521

# (STATE, INPUT) -> NEXT STATE)
def tf(s, t):
    """Transition function for the regex example 'ab' over a Galois ring where p = 65521
    
    Args:
        s : The current state
        t : The input token
    """

    return 2 + 1*t + 10922*s + 54602*s*t + 65520*(s**2) + 65518*(s**2)*t + 54601*(s**3) + 10921*(s**3)*t

alice = VirtualMachine('alice')     # server 0
bob = VirtualMachine('bob')         # server 1
charlie = VirtualMachine('charlie') # client

state = PrivateScalar(0, alice)                 # initial state
shared_state = state.share([alice, bob], PRIME) # initial state shared with servers ()

token = PrivateScalar(1, charlie)               # client inputs new token
shared_token = token.share([alice, bob], PRIME) # client shares token with servers
shared_state = tf(shared_state, shared_token)   # servers compute new state using shares of current state and token



In [26]:

# token = PrivateScalar(1, charlie)
# shared_token = token.share([alice, bob], PRIME)
# shared_state = tf(shared_state, shared_token)

# token = PrivateScalar(1, charlie)
# shared_token = token.share([alice, bob], PRIME)
# shared_state = tf(shared_state, shared_token)
# print(shared_state.reconstruct(alice))